# WorkGraph Tutorial: MD Simulation with DFT Labeling and MLIP Fine-tuning

This tutorial demonstrates building complex workflows with AiiDA WorkGraph through a real-world example: generating DFT-labeled training data and fine-tuning a machine learning interatomic potential (MLIP).

## Workflow Steps

1. **MD Simulation** → Generate diverse NaCl configurations (6 snapshots)
2. **Descriptors** → Calculate MLIP features for filtering (dynamic tasks)
3. **Data Split** → FPS filtering into train/validation/test sets
4. **DFT Labeling** → Quantum ESPRESSO energies, forces, stresses (nested dynamic tasks)
5. **Training Files** → Format ExtXYZ files with DFT labels
6. **Fine-tuning** → Train MACE model on DFT data

## WorkGraph Features

- **Dynamic task generation**: Create tasks at runtime based on trajectory length
- **Nested namespaces**: Organize complex hierarchical outputs
- **Automatic dependencies**: Tasks execute when inputs become available
- **Parallel execution**: Independent calculations run simultaneously

### Load AiiDA Profile

In [1]:
from aiida import load_profile
load_profile()

Profile<uuid='60b17659a9844c4bbd3bef8de0a8f417' name='presto'>

### Verify SSSP Pseudopotentials

Quantum ESPRESSO requires pseudopotentials for DFT calculations. We check that the SSSP library is installed:

In [2]:
from aiida.orm import load_group
try:
    pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
    print(f" SSSP pseudopotentials installed ({len(pseudo_family.nodes)} pseudos)")
except Exception:
    print(" SSSP not installed, Run: aiida-pseudo install sssp -v 1.3 -p efficiency")

 SSSP pseudopotentials installed (103 pseudos)


### Load ML Potential Model
We download a pre-trained MACE model from the janus-core repository. This foundation model will be:
- Used for the initial MD simulation
- Used to calculate descriptors for filtering
- Fine-tuned on our DFT-labeled data

The `ModelData.from_uri()` function automatically caches the model locally to avoid repeated downloads.

In [3]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

### Define Initial Structure

We create a rocksalt NaCl structure with a lattice parameter of 5.63 Å. This serves as the starting point for our MD simulation.

**Why NaCl?** It's a simple ionic crystal that demonstrates:
- Multi-element systems
- Different atomic environments
- Structural variations during MD

In [4]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read, iread

# structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
init_structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

### Load Computational Codes

We need two codes:
- **janus@localhost**: For MLIP calculations (MD, descriptors, training) - runs locally
- **qe@scarf**: For DFT calculations with Quantum ESPRESSO - runs on HPC cluster

See examples>tutorials>aiida_setup for setting up janus code and external computers.

In [5]:
from aiida.orm import load_code
janus_code = load_code("janus@localhost")
qe_code = load_code("qe@scarf")

### Define Calculation Types

We define three calculation types from the `aiida-mlip` plugin:
- `mdCalc`: Molecular dynamics simulation
- `descriptorsCalc`: Compute structural descriptors
- `trainCalc`: Fine-tune the MLIP model

In [6]:
from aiida.plugins import CalculationFactory

mdCalc = CalculationFactory("mlip.md")
descriptorsCalc = CalculationFactory("mlip.descriptors")
trainCalc = CalculationFactory("mlip.train")

# WorkGraph Concepts

## Concept 1: Task Decorators & Dynamic Outputs

WorkGraph provides decorators to convert functions and calculations into workflow tasks. There are three main types:

- `task(CalculationClass)`: Wrap AiiDA calculations
- `@task.calcfunction`: Wrap AiiDA calcfunctions
- `@task.graph`: Create a sub-workflow (task graph)

### Dynamic Task Creation

The `descriptors_task` below demonstrates **dynamic task generation**:
- The `@task.graph` decorator makes this a reusable sub-workflow
- `dynamic(SinglefileData)` in the output namespace means the number of outputs is determined at runtime
- Inside the function, we loop through structures and create a task for each one
- Each task runs independently (in parallel where possible)

**Key WorkGraph Feature**: Notice how we create tasks in a loop and store their outputs in a dictionary. WorkGraph automatically:
1. Tracks dependencies between all generated tasks
2. Executes them in parallel when resources allow
3. Collects all results into the `final_structs` output

## Dynamic Task Creation: Descriptors

**WorkGraph Feature**: The `@task.graph` decorator with `dynamic(SinglefileData)` outputs allows creating variable numbers of tasks at runtime.

**Scientific Purpose**: Calculate MLIP descriptors per structure for filtering diverse configurations. Descriptors characterize local atomic environments and enable intelligent selection of training data.

The loop creates one descriptor calculation per MD snapshot, running in parallel:

In [7]:
from aiida_workgraph import WorkGraph, task, dynamic, namespace
from aiida.orm import SinglefileData

descriptorsTask = task(descriptorsCalc)

@task.graph(outputs=namespace(final_structs=dynamic(SinglefileData)))
def descriptors_task(
    code,
    model,
    device,
    arch,
    file,
):
    results = {}
    
    with file.as_path() as path:
        for i, structs in enumerate(iread(path)):
            structure = StructureData(ase=structs)

            desc_calc = descriptorsTask(
                code=code,
                model=model,
                device=device,
                arch=arch,
                struct=structure,
                calc_per_element=True,
                metadata={"options": {"resources": {"num_machines": 1}}}
            )

            results[f"struct{i}"] = desc_calc.xyz_output

    return {"final_structs": results}

defining outputnode


## Concept 2: Nested Dynamic Tasks & Namespaces

The `qe` task graph demonstrates more advanced WorkGraph features:

### Namespaces for Structured Outputs

```python
qe_output = namespace(trajectory=TrajectoryData, parameters=Dict)
```

This defines a **namespace** - a structured output containing multiple typed fields. Each DFT calculation returns both trajectory and parameters.

### Dynamic Nested Outputs

```python
outputs=namespace(
    test_file=dynamic(qe_output),
    train_file=dynamic(qe_output),
    valid_file=dynamic(qe_output),
)
```

This creates a **nested dynamic structure**:
- Three fixed categories (test, train, valid)
- Each category contains a dynamic number of results
- Each result is a namespace with trajectory and parameters

**WorkGraph's Power**: This task processes three input files and creates dozens of DFT calculations. WorkGraph:
1. Automatically parallelizes all independent calculations
2. Organizes outputs hierarchically 
3. Passes the entire structure to downstream tasks
4. Handles remote execution (HPC cluster) transparently

In [8]:
from aiida.orm import (
    TrajectoryData,
    StructureData, 
    load_group, 
    KpointsData, 
    InstalledCode, 
    List, 
    Dict,
)
from pathlib import Path
import yaml
from aiida_quantumespresso.calculations.pw import PwCalculation

PwTask = task(PwCalculation)
qe_output = namespace(trajectory=TrajectoryData, parameters=Dict)

@task.graph(
    outputs=namespace(
        test_file=dynamic(qe_output),
        train_file=dynamic(qe_output),
        valid_file=dynamic(qe_output),
    )
)
def qe(
    code: InstalledCode,
    kpoints_mesh: List,
    task_metadata: Dict,
    test_file: SinglefileData,
    train_file: SinglefileData,
    valid_file: SinglefileData
    ):

    kpoints = KpointsData()
    kpoints.set_kpoints_mesh(kpoints_mesh)

    pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
    
    files = {"test_file": test_file, "train_file": train_file, "valid_file": valid_file}
    results = {"test_file": {}, "train_file": {}, "valid_file": {}}
    
    for file_name, file in files.items():
        with file.as_path() as path:
            for i, structs in enumerate(iread(path, format="extxyz")):
                
                structure = StructureData(ase=structs)
                pseudos = pseudo_family.get_pseudos(structure=structure)

                ecutwfc, ecutrho = pseudo_family.get_recommended_cutoffs(
                    structure=structure,
                    unit='Ry',
                )

                pw_params = {
                    "CONTROL": {
                        "calculation": "scf",
                        'tprnfor': True,
                        'tstress': True,
                    },
                    "SYSTEM": {
                        "ecutwfc": ecutwfc,
                        "ecutrho": ecutrho,
                    },
                }
                
                qe_task = PwTask(
                    code=code,
                    parameters=pw_params,
                    kpoints=kpoints,
                    pseudos=pseudos,
                    metadata=task_metadata.value,
                    structure=structure,
                )
                
                structfile = f"{file_name}.struct{i}"

               
                results[file_name][f"struct_{i}"] = {
                    "trajectory": qe_task.output_trajectory,
                    "parameters": qe_task.output_parameters,
                }
    return results

## Concept 3: CalcFunction Tasks

The `@task.calcfunction` decorator wraps AiiDA calcfunctions - Python functions that:
- Are tracked in the provenance graph
- Have serializable inputs and outputs  
- Can be restarted if they fail

**WorkGraph Integration**: CalcFunction tasks can:
- Receive dynamic outputs from upstream tasks (like the QE results)
- Process complex nested data structures
- Return AiiDA data types that flow to downstream tasks

The `create_train_file` task receives the entire nested QE output structure and processes it, demonstrating how WorkGraph handles complex data flow between tasks.

In [9]:
from aiida_mlip.data.config import JanusConfigfile
from aiida.orm import Dict
from ase.io import write
from ase import units

@task.calcfunction(outputs = ["JanusConfigfile"])
def create_train_file(**inputs):

    training_files = {}
    
    for file_name, structs in inputs.items():
        path = Path(f"mlip_{file_name}.extxyz")

        for struct_out_params in structs.values():
            
            trajectory = struct_out_params["trajectory"]

            fileStructure = trajectory.get_structure(index=0)
            fileAtoms = fileStructure.get_ase()

            stress = trajectory.arrays["stress"][0]
            converted_stress = stress * units.GPa
            fileAtoms.info["qe_stress"] = converted_stress

            fileAtoms.info["units"] = {
                "energy": "eV",
                "forces": "ev/Ang",
                "stress": "ev/Ang^3"
            }
            fileAtoms.set_array("qe_forces", trajectory.arrays["forces"][0])

            parameters = struct_out_params["parameters"]
            fileParams = parameters.get_dict()
            fileAtoms.info["qe_energy"] = fileParams["energy"]
            write(path, fileAtoms, append=True)

        training_files[file_name] = str(path.resolve())

    with open("JanusConfigfile.yml", "a") as f:
        yaml.safe_dump(training_files, f, sort_keys=False)

    return{'JanusConfigfile': JanusConfigfile(Path("JanusConfigfile.yml").resolve())}

## Data Processing Tasks

This calcfunction task demonstrates how WorkGraph handles data transformations between computational tasks. It takes dynamic outputs from the descriptor calculation and produces structured files for the next stage.

In [10]:
from aiida.orm import Int
from sample_split import process_and_split_data

@task.calcfunction(outputs = ["test_file", "train_file", "valid_file"])
def create_qe_files(**inputs):

    n_samples = Int(len(inputs['trajectory_data']))
    print(n_samples)

    files = process_and_split_data( 
        **inputs,
        n_samples=n_samples,
    )

    return {
        "train_file": SinglefileData(files["train_file"]),
        "test_file": SinglefileData(files["test_file"]),
        "valid_file": SinglefileData(files["valid_file"])
    }

## Configure Input Parameters


**NVT Ensemble**: 
- Maintains constant Number of particles, Volume, and Temperature
- Suitable for sampling structural variations
- Uses a thermostat to control temperature

See https://stfc.github.io/janus-core/tutorials/cli/md.html for other ensembles.

In [11]:
from aiida.orm import Str, Float, Bool, Dict
inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "device": Str("cpu"),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "ensemble": Str("NVT"),
    "struct": init_structure,
    "md_kwargs": Dict(
        {
            "steps": 10,
            "traj-every": 2
        }
    )
}
calc_inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "device": Str("cpu"),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
}
split_task_inputs = {
    "config_types": Str(""),
    "prefix": Str(""),
    "scale": Float(1.0e5),
    "append_mode": Bool(False),
}
qe_inputs = {
    "task_metadata": Dict({
            "options": {
                "resources": {
                    "num_machines": 1,
                    "num_mpiprocs_per_machine": 32,
                },
                "max_wallclock_seconds": 3600,
                "queue_name": "scarf",
                "qos": "scarf",
                "environment_variables": {},
                "withmpi": True,
                "prepend_text": """
                    module purge
                    module use /work4/scd/scarf562/eb-common/modules/all
                    module load amd-modules
                    module load QuantumESPRESSO/7.2-foss-2023a
                """,
                "append_text": "",
            },
    }),
    "kpoints_mesh": List([1, 1, 1]),
    "code": qe_code,
}

## Concept 4: Assembling the WorkGraph

Now we combine all tasks into a single workflow. The `@task.graph()` decorator creates a top-level workflow function.

### Key WorkGraph Features Demonstrated:

1. **Task Creation**: Convert calculations to tasks
   ```python
   mdTask = task(mdCalc)
   trainTask = task(trainCalc)
   ```

2. **Automatic Dependency Resolution**: WorkGraph analyzes the data flow:
   - `md_calc` runs first (no dependencies)
   - `descriptors_calc` waits for `md_calc.traj_file`
   - `split_task` waits for `descriptors_calc.final_structs`
   - `qe_task` waits for split outputs
   - Tasks run in parallel when possible

3. **Output Linking**: Connect task outputs to downstream inputs:
   ```python
   file=md_calc.traj_file  # Output from md_calc flows to descriptors_task
   ```

4. **Dynamic Data Flow**: The workflow handles:
   - Variable numbers of structures from MD
   - Dynamic descriptor calculations  
   - Nested QE outputs
   - All without knowing exact counts ahead of time

5. **Execution Control**: WorkGraph manages:
   - Local vs. remote execution
   - Resource allocation
   - Failure handling
   - Provenance tracking

In [12]:
mdTask = task(mdCalc)
trainTask = task(trainCalc)

@task.graph()
def md_training_workflow(
    janus_code,
    qe_code,
    model,
    md_kwargs,
    ensemble,
    init_structure,
    split_inputs,
    qe_metadata,
    qe_kpoints,
):
    md_calc = mdTask(
        code=janus_code,
        model=model,
        arch=Str(model.architecture),
        device=Str("cpu"),
        metadata={"options": {"resources": {"num_machines": 1}}},
        ensemble=ensemble,
        struct=init_structure,
        md_kwargs=md_kwargs
    )

    descriptors_calc = descriptors_task(
        code=janus_code,
        model=model,
        device=Str("cpu"),
        arch=Str(model.architecture),
        file=md_calc.traj_file
    )
   
    split_task = create_qe_files(
        **split_inputs,
        trajectory_data=descriptors_calc.final_structs,
    )
    
    qe_task = qe(
        code=qe_code,
        kpoints_mesh=qe_kpoints,
        task_metadata=qe_metadata,
        test_file=split_task.test_file,
        train_file=split_task.train_file,
        valid_file=split_task.valid_file,
    )

    training_files = create_train_file(
        test_file=qe_task.test_file,
        train_file=qe_task.train_file,
        valid_file=qe_task.valid_file,
    )

    train_task = trainTask(
        mlip_config=training_files.JanusConfigfile,
        code=janus_code,
        foundation_model=model,
        metadata={"options": {"resources": {"num_machines": 1}}},
        fine_tune=True,
    )

### Building the WorkGraph

The `.build()` method:
1. Creates the workflow instance
2. Analyzes task dependencies
3. Prepares for execution
4. Returns a WorkGraph object you can inspect, visualize, or run

In [13]:
wg = md_training_workflow.build(
    janus_code=janus_code,
    qe_code=qe_code,
    model=model,
    md_kwargs=inputs["md_kwargs"],
    ensemble=inputs["ensemble"],
    init_structure=init_structure,
    split_inputs=split_task_inputs,
    qe_metadata=qe_inputs["task_metadata"],
    qe_kpoints=qe_inputs["kpoints_mesh"],
)

### Execution

`wg.run()` submits the workflow. WorkGraph:
- Executes tasks as their dependencies become available
- Runs independent tasks in parallel
- Handles local and remote execution
- Stores all data in AiiDA's database
- Tracks full provenance automatically

In [14]:
wg.run()

12/24/2025 10:06:56 AM <916425> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
12/24/2025 10:06:56 AM <916425> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
12/24/2025 10:06:56 AM <916425> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
12/24/2025 10:06:57 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7535|WorkGraphEngine|continue_workgraph]: tasks ready to run: MD
12/24/2025 10:06:57 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7535|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 7538


/home/qoj42292/.aiida/scratch/presto/e6/99/6657-765e-4311-bf7a-a6f8dec4d5ae md-summary.yml


12/24/2025 10:07:13 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7535|WorkGraphEngine|update_task_state]: Task: MD, type: CALCJOB, finished.
12/24/2025 10:07:15 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7535|WorkGraphEngine|continue_workgraph]: tasks ready to run: descriptors_task
12/24/2025 10:07:17 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7535|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 7555
12/24/2025 10:07:18 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7555|WorkGraphEngine|continue_workgraph]: tasks ready to run: Descriptors,Descriptors1,Descriptors2,Descriptors3,Descriptors4,Descriptors5
12/24/2025 10:07:21 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7555|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 7560, 7565, 7570, 7575, 7580, 7585
12/24/2025 10:

uuid: da09e2c1-ac92-4b8d-8fea-a0db15725209 (unstored) value: 6
create files: train_file=PosixPath('train.xyz'), valid_file=PosixPath('valid.xyz') and test_file=PosixPath('test.xyz')
Processing: ('all', 'aiida'), 6 frames
  ('all', 'aiida'): total=6, train_target=4,                     vt_target=2


/home/qoj42292/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 6
  warnings.warn(f"k is too large, set to {n_pts}")
/home/qoj42292/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 2
  warnings.warn(f"k is too large, set to {n_pts}")
12/24/2025 10:08:05 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7535|WorkGraphEngine|update_task_state]: Task: create_qe_files, type: CALCFUNCTION, finished.
12/24/2025 10:08:05 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7535|WorkGraphEngine|continue_workgraph]: tasks ready to run: qe
12/24/2025 10:08:06 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7535|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 7633
12/24/2025 10:08:06 AM <916425> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7633|WorkGra

{}

### Visualize the WorkGraph

WorkGraph provides an interactive visualization showing:
- All tasks and their types
- Dependencies between tasks  
- Execution status
- Input/output connections

This helps you understand the workflow structure and debug issues.

In [16]:
wg

NodeGraphWidget(positions={'Train': [2261.4500000000003, 12]}, settings={'minimap': True}, states={'graph_inpu…

## Summary: What You Learned

### WorkGraph Concepts:
1. **Task Decorators**: `task()`, `@task.calcfunction`, `@task.graph` for different task types
2. **Dynamic Outputs**: Creating tasks at runtime with `dynamic()`
3. **Namespaces**: Organizing structured outputs with `namespace()`
4. **Data Flow**: Linking task outputs to inputs automatically
5. **Dependency Management**: WorkGraph automatically orders tasks based on data dependencies
6. **Parallel Execution**: Independent tasks run simultaneously
7. **Provenance**: Full tracking of all calculations and data

### Workflow Results:
- Fine-tuned MACE model
- Training datasets with DFT labels
- Complete AiiDA provenance for reproducibility

### Next Steps:
- Try modifying task parameters
- Add new tasks to the workflow
- Explore the AiiDA provenance graph
- Create your own WorkGraph workflows!